In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from requests import get
from lxml import html
from selenium import webdriver
import time
import re
from selenium import webdriver
import chromedriver_autoinstaller
import mysql.connector
from mysql.connector import Error
import os

chromedriver_autoinstaller.install()  

def removeChar(s,x):
    return s.replace(x, "")

In [2]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_autoinstaller
import os
import time

# Auto-install Chrome driver
chromedriver_autoinstaller.install()

# Helper function to remove a specific substring
def removeChar(s, x):
    return s.replace(x, "")

# User input for the car they want to buy
user = input("Enter the name of the Car You want to buy: ")

# Prepare the search query by replacing spaces with "+"
st = "+".join(user.split())

# Initialize the Chrome driver once (outside the loop)
driver = webdriver.Chrome()

# Initialize variables
page = 1
total_count = 0

while True:
    url = f'https://www.pakwheels.com/used-cars/search/-/?page={page}&q={st}'
    print("Link:", url, "\n")

    # Open the page in the browser
    driver.get(url)
    time.sleep(2)  # Adding a small delay for the page to fully load

    # Parse the page content
    html = driver.page_source
    parsed_html = BeautifulSoup(html, "html.parser")

    # Scrape car listings
    containers = parsed_html.find_all("div", {"class": "col-md-9 grid-style"})

    if not containers:
        print("No more results, ending scraping.")
        break

    count = 0
    name, city, info1, info2, info3, info4, info5, price, certified, managed_by = [], [], [], [], [], [], [], [], [], []

    for container in containers:
        count += 1
        car_name = container.find('h3').text
        car_name = removeChar(car_name, " for Sale")
        name.append(car_name)

        city_name = container.find('ul', class_='list-unstyled search-vehicle-info fs13').text
        city_name = re.sub(r'[^a-zA-Z]', '', city_name)
        city.append(city_name)

        # Fetch car details (year, KMs, fuel type, etc.)
        info_car = container.find('ul', class_='list-unstyled search-vehicle-info-2 fs13').text.split()
        info1.append(int(info_car[0]))  # year
        info2.append(info_car[1] + info_car[2])  # KMs
        info3.append(info_car[3])  # fuel
        info4.append(info_car[4] + info_car[5])  # horsepower
        info5.append(info_car[6])  # transmission

        # Fetch car price
        prices = container.find('div', class_='price-details generic-dark-grey').text
        prices = re.sub(r'\s+', ' ', prices.strip())
        price.append(prices)

        # Certified or not
        cert = container.find(class_='pull-left mr5')
        certified.append("Certified" if cert else "Not Certified")

        # Managed by PakWheels or Customer
        manage = container.find('span', class_='sold-by-pw')
        managed_by.append("PakWheels" if manage else "Customer")

    # Convert price from text to numbers
    price_number = []
    for price_num in price:
        check = price_num.find('PKR')
        if check == -1:
            price_number.append(0)
        else:
            price_num = price_num.replace('PKR', '')
            number, unit = price_num.split()
            number = float(number)

            if unit == 'lacs':
                number *= 100000
            elif unit == 'crore':
                number *= 10000000

            price_number.append(int(number))

    # Create a DataFrame for this page
    df = pd.DataFrame(list(zip(name, price, price_number, city, info1, info2, info3, info4, info5, certified, managed_by)),
                      columns=['Name', 'PriceInText', 'Price', 'City', 'Year', 'Distance', 'Fuel', 'HorsePower', 'Transmission', 'Certified', 'ManagedBy'])
    df['Distance2'] = df['Distance'].str.replace(',', '').str.replace('km', '')
    df['Distance2'] = pd.to_numeric(df['Distance2'])

    # Check if Excel file exists, if so, append data; otherwise create a new file
    if os.path.exists("Results.xlsx"):
        data2 = pd.read_excel("Results.xlsx")
        data1 = pd.concat([df, data2], ignore_index=True)
        data1 = data1[data1["HorsePower"].str.contains("0.0kWh") == False]
        data1 = data1[data1["PriceInText"].str.contains("Call") == False]
        data1.drop_duplicates(inplace=True)
        data1['Fuel'] = data1['Fuel'].replace('CNG', 'Petrol')
        data1.to_excel("Results.xlsx", index=False, sheet_name='Used Cars')
    else:
        df.to_excel("Results.xlsx", index=False, sheet_name='Used Cars')

    page += 1
    total_count += count

    # Check if there's a next page
    find_next_page = parsed_html.find("li", {"class": "next_page"})
    if not find_next_page:
        print('No more Pages left')
        break

# Close the browser after all pages are scraped
driver.quit()

Enter the name of the Car You want to buy:  


Link: https://www.pakwheels.com/used-cars/search/-/?page=1&q= 

Link: https://www.pakwheels.com/used-cars/search/-/?page=2&q= 

Link: https://www.pakwheels.com/used-cars/search/-/?page=3&q= 

Link: https://www.pakwheels.com/used-cars/search/-/?page=4&q= 

Link: https://www.pakwheels.com/used-cars/search/-/?page=5&q= 

Link: https://www.pakwheels.com/used-cars/search/-/?page=6&q= 

Link: https://www.pakwheels.com/used-cars/search/-/?page=7&q= 

Link: https://www.pakwheels.com/used-cars/search/-/?page=8&q= 

Link: https://www.pakwheels.com/used-cars/search/-/?page=9&q= 

Link: https://www.pakwheels.com/used-cars/search/-/?page=10&q= 

Link: https://www.pakwheels.com/used-cars/search/-/?page=11&q= 

Link: https://www.pakwheels.com/used-cars/search/-/?page=12&q= 

Link: https://www.pakwheels.com/used-cars/search/-/?page=13&q= 

Link: https://www.pakwheels.com/used-cars/search/-/?page=14&q= 

Link: https://www.pakwheels.com/used-cars/search/-/?page=15&q= 

Link: https://www.pakwheels.com/us

KeyboardInterrupt: 

In [4]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller
import os
import time

# Auto-install Chrome driver
chromedriver_autoinstaller.install()

# Helper function to remove a specific substring
def removeChar(s, x):
    return s.replace(x, "")

# Helper function to convert price text to number
def convert_price(price_str):
    if 'PKR' not in price_str:
        return 0
    price_str = price_str.replace('PKR', '').strip()
    number, unit = price_str.split()
    number = float(number.replace(',', ''))  # Handle commas in the number
    if unit == 'lacs':
        return int(number * 100000)
    elif unit == 'crore':
        return int(number * 10000000)
    return 0

# User input for the car they want to buy
user = input("Enter the name of the Car You want to buy: ")

# Prepare the search query by replacing spaces with "+"
st = "+".join(user.split())

# Initialize the Chrome driver
driver = webdriver.Chrome()

# Initialize variables
page = 1
total_count = 0
all_data = []

try:
    while True:
        url = f'https://www.pakwheels.com/used-cars/search/-/?page={page}&q={st}'
        print("Scraping URL:", url)

        # Open the page in the browser
        driver.get(url)

        # Wait until the car listings are visible (improves reliability)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'col-md-9.grid-style'))
        )

        # Parse the page content
        html = driver.page_source
        parsed_html = BeautifulSoup(html, "html.parser")

        # Scrape car listings
        containers = parsed_html.find_all("div", {"class": "col-md-9 grid-style"})

        if not containers:
            print("No more results, ending scraping.")
            break

        count = 0
        name, city, info1, info2, info3, info4, info5, price, certified, managed_by = [], [], [], [], [], [], [], [], [], []

        for container in containers:
            count += 1
            car_name = container.find('h3').text
            car_name = removeChar(car_name, " for Sale")
            name.append(car_name)

            city_name = container.find('ul', class_='list-unstyled search-vehicle-info fs13').text
            city_name = re.sub(r'[^a-zA-Z]', '', city_name)
            city.append(city_name)

            # Fetch car details (year, KMs, fuel type, etc.)
            info_car = container.find('ul', class_='list-unstyled search-vehicle-info-2 fs13').text.split()
            info1.append(int(info_car[0]))  # year
            info2.append(info_car[1] + info_car[2])  # KMs
            info3.append(info_car[3])  # fuel
            info4.append(info_car[4] + info_car[5])  # horsepower
            info5.append(info_car[6])  # transmission

            # Fetch car price
            prices = container.find('div', class_='price-details generic-dark-grey').text.strip()
            price.append(prices)

            # Certified or not
            cert = container.find(class_='pull-left mr5')
            certified.append("Certified" if cert else "Not Certified")

            # Managed by PakWheels or Customer
            manage = container.find('span', class_='sold-by-pw')
            managed_by.append("PakWheels" if manage else "Customer")

        # Convert price from text to numbers
        price_number = [convert_price(p) for p in price]

        # Create a DataFrame for this page
        df = pd.DataFrame(list(zip(name, price, price_number, city, info1, info2, info3, info4, info5, certified, managed_by)),
                          columns=['Name', 'PriceInText', 'Price', 'City', 'Year', 'Distance', 'Fuel', 'HorsePower', 'Transmission', 'Certified', 'ManagedBy'])

        df['Distance2'] = df['Distance'].str.replace(',', '').str.replace('km', '')
        df['Distance2'] = pd.to_numeric(df['Distance2'])

        all_data.append(df)

        page += 1
        total_count += count

        # Check if there's a next page
        find_next_page = parsed_html.find("li", {"class": "next_page"})
        if not find_next_page:
            print('No more Pages left')
            break

finally:
    driver.quit()

# Combine all pages' data
final_df = pd.concat(all_data, ignore_index=True)
final_df = final_df[final_df["HorsePower"].str.contains("0.0kWh") == False]
final_df = final_df[final_df["PriceInText"].str.contains("Call") == False]
final_df.drop_duplicates(inplace=True)
final_df['Fuel'] = final_df['Fuel'].replace('CNG', 'Petrol')

# Save to CSV (create or update file)
if os.path.exists("AllScrapDataFile.csv"):
    old_data = pd.read_csv("AllScrapDataFile.csv")
    combined_data = pd.concat([old_data, final_df], ignore_index=True).drop_duplicates()
    combined_data.to_csv("AllScrapDataFile.csv", index=False)
else:
    final_df.to_csv("AllScrapDataFile.csv", index=False)

print(f"Scraping complete. {total_count} records saved to Results.csv.")

Enter the name of the Car You want to buy:  


Scraping URL: https://www.pakwheels.com/used-cars/search/-/?page=1&q=
Scraping URL: https://www.pakwheels.com/used-cars/search/-/?page=2&q=
Scraping URL: https://www.pakwheels.com/used-cars/search/-/?page=3&q=
Scraping URL: https://www.pakwheels.com/used-cars/search/-/?page=4&q=
Scraping URL: https://www.pakwheels.com/used-cars/search/-/?page=5&q=
Scraping URL: https://www.pakwheels.com/used-cars/search/-/?page=6&q=


KeyboardInterrupt: 